<a href="https://colab.research.google.com/github/anumsajid13/Fabricae-FYP/blob/main/flux.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/ostris/ai-toolkit
!mkdir -p /content/dataset

Cloning into 'ai-toolkit'...
remote: Enumerating objects: 3810, done.
remote: Counting objects: 100% (3809/3809), done.
remote: Compressing objects: 100% (926/926), done.
remote: Total 3810 (delta 2905), reused 3665 (delta 2796), pack-reused 1 (from 1)
Receiving objects: 100% (3810/3810), 29.68 MiB | 38.09 MiB/s, done.
Resolving deltas: 100% (2905/2905), done.


In [ ]:
!cd ai-toolkit && git submodule update --init --recursive && pip install -r requirements.txt

Submodule 'repositories/batch_annotator' (https://github.com/ostris/batch-annotator) registered for path 'repositories/batch_annotator'
Submodule 'repositories/ipadapter' (https://github.com/tencent-ailab/IP-Adapter.git) registered for path 'repositories/ipadapter'
Submodule 'repositories/leco' (https://github.com/p1atdev/LECO) registered for path 'repositories/leco'
Submodule 'repositories/sd-scripts' (https://github.com/kohya-ss/sd-scripts.git) registered for path 'repositories/sd-scripts'
Cloning into '/content/ai-toolkit/repositories/batch_annotator'...
Cloning into '/content/ai-toolkit/repositories/ipadapter'...
Cloning into '/content/ai-toolkit/repositories/leco'...
Cloning into '/content/ai-toolkit/repositories/sd-scripts'...
Submodule path 'repositories/batch_annotator': checked out '420e142f6ad3cc14b3ea0500affc2c6c7e7544bf'
Submodule 'repositories/controlnet' (https://github.com/lllyasviel/ControlNet-v1-1-nightly.git) registered for path 'repositories/batch_annotator/repositor

In [ ]:
import getpass
import os


hf_token = getpass.getpass('Enter your HF access token and press enter: ')


os.environ['HF_TOKEN'] = hf_token

print("HF_TOKEN environment variable has been set.")

Enter your HF access token and press enter: ··········
HF_TOKEN environment variable has been set.


In [ ]:

import os
import sys
sys.path.append('/content/ai-toolkit')
from toolkit.job import run_job
from collections import OrderedDict
from PIL import Image
import os
os.environ["HF_HUB_ENABLE_HF_TRANSFER"] = "1"



In [ ]:
import os
import shutil
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')


image_folder = '/content/drive/MyDrive/Create_Dataset/Create_Dataset/Resized_Images'
prompt_folder = '/content/drive/MyDrive/Create_Dataset/Create_Dataset/textfilekeyword_folder'

combined_folder = '/content/dataset'


os.makedirs(combined_folder, exist_ok=True)

image_files = os.listdir(image_folder)
text_files = os.listdir(prompt_folder)

for image_file in image_files:

    text_file = os.path.splitext(image_file)[0] + '.txt'

    if text_file in text_files:

        image_path = os.path.join(image_folder, image_file)
        text_path = os.path.join(prompt_folder, text_file)

        shutil.copy(image_path, combined_folder)
        shutil.copy(text_path, combined_folder)
        print(f'Copied: {image_file} and {text_file} to {combined_folder}')
    else:
        print(f'No corresponding text file for: {image_file}')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Copied: image107.png and image107.txt to /content/dataset
Copied: image1.png and image1.txt to /content/dataset
Copied: image102.png and image102.txt to /content/dataset
Copied: image105.png and image105.txt to /content/dataset
Copied: image103.png and image103.txt to /content/dataset
Copied: image106.png and image106.txt to /content/dataset
Copied: image101.png and image101.txt to /content/dataset
Copied: image10.png and image10.txt to /content/dataset
Copied: image100.png and image100.txt to /content/dataset
Copied: image104.png and image104.txt to /content/dataset
Copied: image108.png and image108.txt to /content/dataset
Copied: image116.png and image116.txt to /content/dataset
Copied: image121.png and image121.txt to /content/dataset
Copied: image109.png and image109.txt to /content/dataset
Copied: image115.png and image115.txt to /content/dataset
Copied:

In [ ]:
import torch
torch.cuda.empty_cache()


In [ ]:
from diffusers import AutoPipelineForText2Image
import torch

pipeline = AutoPipelineForText2Image.from_pretrained("black-forest-labs/FLUX.1-dev", torch_dtype=torch.bfloat16)

pipeline.enable_model_cpu_offload()
pipeline.enable_sequential_cpu_offload()
pipeline.vae.enable_slicing()
pipeline.vae.enable_tiling()
pipeline.to(torch.float16)



Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

FluxPipeline {
  "_class_name": "FluxPipeline",
  "_diffusers_version": "0.31.0.dev0",
  "_name_or_path": "black-forest-labs/FLUX.1-dev",
  "scheduler": [
    "diffusers",
    "FlowMatchEulerDiscreteScheduler"
  ],
  "text_encoder": [
    "transformers",
    "CLIPTextModel"
  ],
  "text_encoder_2": [
    "transformers",
    "T5EncoderModel"
  ],
  "tokenizer": [
    "transformers",
    "CLIPTokenizer"
  ],
  "tokenizer_2": [
    "transformers",
    "T5TokenizerFast"
  ],
  "transformer": [
    "diffusers",
    "FluxTransformer2DModel"
  ],
  "vae": [
    "diffusers",
    "AutoencoderKL"
  ]
}

In [ ]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"


In [ ]:
from collections import OrderedDict

job_to_run = OrderedDict([
    ('job', 'extension'),
    ('config', OrderedDict([
       
        ('name', 'flux_dev_finetune'),
        ('process', [
            OrderedDict([
                ('type', 'sd_trainer'),
                # root folder to save training sessions/samples/weights
                ('training_folder', '/content/output'),
                # uncomment to see performance stats in the terminal every N steps
                ('performance_log_every', 50),
                ('device',  'cuda:0'),
               
                ('network', OrderedDict([
                    ('type', 'lora'),
                    ('linear', 16),
                    ('linear_alpha', 32)
                ])),
                ('save', OrderedDict([
                    ('dtype', 'float16'),  # precision to save
                    ('save_every', 250),  # save every this many steps
                    ('max_step_saves_to_keep', 4)  # how many intermittent saves to keep
                ])),
                ('datasets', [
                    # datasets are a folder of images. captions need to be txt files with the same name as the image
                    # for instance image2.jpg and image2.txt. Only jpg, jpeg, and png are supported currently
                    # images will automatically be resized and bucketed into the resolution specified
                    OrderedDict([
                        ('folder_path', '/content/dataset'),
                        ('caption_ext', 'txt'),
                        ('caption_dropout_rate', 0.05),  # will drop out the caption 5% of time
                        ('shuffle_tokens', True),  # shuffle caption order, split by commas
                        ('cache_latents_to_disk', True), 
                        ('resolution', [512])  # flux enjoys multiple resolutions
                    ])
                ]),
                ('train', OrderedDict([
                    ('batch_size', 8),
                    ('steps', 1000),  # total number of steps to train 500 - 4000 is a good range
                    ('gradient_accumulation_steps', 2),
                    ('train_unet', True),
                    ('train_text_encoder', False),  # probably won't work with flux
                    ('content_or_style', 'balanced'),  # content, style, balanced
                    ('gradient_checkpointing', True),  # need the on unless you have a ton of vram
                    ('noise_scheduler', 'flowmatch'),  # for training only
                    ('optimizer', 'adamw'),
                    ('lr', 1e-4),

                

                    # ema will smooth out learning, but could slow it down. Recommended to leave on.
                    ('ema_config', OrderedDict([
                        ('use_ema', True),
                        ('ema_decay', 0.99)
                    ])),

                    # will probably need this if gpu supports it for flux, other dtypes may not work correctly
                    ('dtype', 'float16')
                ])),
                ('model', OrderedDict([
                    # huggingface model name or path
                    ('name_or_path', 'black-forest-labs/FLUX.1-dev'),
                    ('is_flux', True),
                    ('quantize', True),  # run 8bit mixed precision
                    #('low_vram', True),  # uncomment this if the GPU is connected to your monitors. It will use less vram to quantize, but is slower.
                ])),
                ('sample', OrderedDict([
                    ('sampler', 'flowmatch'),  # must match train.noise_scheduler
                    ('sample_every', 500),  # sample every this many steps
                    ('width', 512),
                    ('height', 512),
                    ('prompts', [
                       
                        'An intricate floral motif inspired by traditional Mughal art, with ornate paisleys and delicate vines in shades of deep maroon, emerald green, and gold on a cream background, reminiscent of classic shalwar kameez patterns.',
                        'A luxurious pattern featuring block-printed motifs of lotus flowers and mandalas in muted pastel tones of pink, peach, and mint green, evoking the rich textile traditions of Pakistans artisanal heritage.',
                        'A sophisticated fusion of contemporary minimalism with traditional Pakistani embroidery, featuring delicate floral butis scattered over a soft linen base in pastel tones, perfect for summer tunics.',
                        'A sophisticated fusion of contemporary minimalism with traditional Pakistani embroidery, featuring delicate floral butis scattered over a soft linen base in pastel tones, perfect for summer tunics.',
                        'A Balochi-style geometric pattern with diamond and triangle shapes in earthy tones of rust, brown, and cream, paying homage to the nomadic craftsmanship of Pakistans western regions',
                        'A playful polka dot pattern with oversized, irregular dots in bold primary colors like red, blue, and yellow, set against a crisp white background, giving a retro yet fresh vibe for summer dresses',
                        'A sophisticated tartan pattern in deep shades of forest green, navy, and burgundy, perfect for stylish woolen skirts and scarves, channeling a refined Scottish heritage look.',
                        'A Kashmiri shawl-inspired pattern featuring delicate swirls of vines and paisleys in soft pastel shades, woven into an intricate border design for a light, airy, and elegant fabric.',
                        'a serene mountain landscape pattern with stylized pine trees, snow-capped peaks, and tranquil lakes, in earthy tones and soft pastels, creating a calming atmosphere',
                        'A hand-painted style floral pattern inspired by the phulkari art of Punjab, with large blooming flowers in bright shades of fuchsia, orange, and yellow, set against a neutral beige background.'
                    ]),
                    ('neg', ''),  # not used on flux
                    ('seed', 42),
                    ('walk_seed', True),
                    ('guidance_scale', 4),
                    ('sample_steps', 20)
                ]))
            ])
        ])
    ])),
    
    ('meta', OrderedDict([
        ('name', '[name]'),
        ('version', '1.0')
    ]))
])


In [ ]:
run_job(job_to_run)

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/xformers/__init__.py", line 57, in _is_triton_available
    import triton  # noqa
ModuleNotFoundError: No module named 'triton'
/usr/local/lib/python3.10/dist-packages/controlnet_aux/mediapipe_face/mediapipe_face_common.py:7: UserWarning: The module 'mediapipe' is not installed. The package will have limited functionality. Please install it using the command: pip install 'mediapipe'
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/controlnet_aux/segment_anything/modeling/tiny_vit_sam.py:654: UserWarning: Overwriting tiny_vit_5m_224 in registry with controlnet_aux.segment_anything.modeling.tiny_vit_sam.tiny_vit_5m_224. This is because the name being registered conflicts with an existing name. Please check if this is not expected.
  return register_model(fn_wrapper)
/usr/local/lib/python3.10/dist-packages/controlnet_aux/segment_anything/modeling/tiny_vit_sam.py:654: UserWarning: Overwriting tiny_vi

{
    "type": "sd_trainer",
    "training_folder": "/content/output",
    "performance_log_every": 50,
    "device": "cuda:0",
    "network": {
        "type": "lora",
        "linear": 16,
        "linear_alpha": 32
    },
    "save": {
        "dtype": "float16",
        "save_every": 250,
        "max_step_saves_to_keep": 4
    },
    "datasets": [
        {
            "folder_path": "/content/dataset",
            "caption_ext": "txt",
            "caption_dropout_rate": 0.05,
            "shuffle_tokens": true,
            "cache_latents_to_disk": true,
            "resolution": [
                512
            ]
        }
    ],
    "train": {
        "batch_size": 8,
        "steps": 1000,
        "gradient_accumulation_steps": 2,
        "train_unet": true,
        "train_text_encoder": false,
        "content_or_style": "balanced",
        "gradient_checkpointing": true,
        "noise_scheduler": "flowmatch",
        "optimizer": "adamw",
        "lr": 0.0001,
        "ema_

scheduler/scheduler_config.json:   0%|          | 0.00/274 [00:00<?, ?B/s]

Loading vae


vae/config.json:   0%|          | 0.00/774 [00:00<?, ?B/s]

...


Timer 'flux_dev_finetune Timer':
 - 7.0449s avg - train_loop, num = 10
 - 4.3523s avg - backward, num = 10
 - 1.9977s avg - predict_unet, num = 10
 - 0.2975s avg - calculate_loss, num = 10
 - 0.2331s avg - encode_prompt, num = 10
 - 0.1475s avg - optimizer_step, num = 10
 - 0.2248s avg - reset_batch, num = 1
 - 0.0055s avg - get_batch, num = 10
 - 0.0031s avg - preprocess_batch, num = 10
 - 0.0021s avg - prepare_noise, num = 10
 - 0.0008s avg - prepare_latents, num = 10
 - 0.0006s avg - batch_cleanup, num = 10
 - 0.0000s avg - scheduler_step, num = 10
 - 0.0000s avg - prepare_prompt, num = 10
 - 0.0000s avg - grad_setup, num = 10
 - 0.0000s avg - log_to_tensorboard, num = 1



flux_dev_finetune:  95%|█████████▍| 949/1000 [1:51:04<05:58,  7.02s/it, lr: 1.0e-04 loss: 3.271e-01]


Timer 'flux_dev_finetune Timer':
 - 7.0730s avg - train_loop, num = 10
 - 4.3508s avg - backward, num = 10
 - 2.0025s avg - predict_unet, num = 10
 - 0.2779s avg - calculate_loss, num = 10
 - 0.2354s avg - encode_prompt, num = 10
 - 0.1480s avg - optimizer_step, num = 10
 - 0.2128s avg - reset_batch, num = 1
 - 0.0095s avg - get_batch, num = 10
 - 0.0034s avg - preprocess_batch, num = 10
 - 0.0022s avg - prepare_noise, num = 10
 - 0.0009s avg - prepare_latents, num = 10
 - 0.0008s avg - batch_cleanup, num = 10
 - 0.0000s avg - grad_setup, num = 10
 - 0.0000s avg - scheduler_step, num = 10
 - 0.0000s avg - prepare_prompt, num = 10



flux_dev_finetune: 100%|█████████▉| 999/1000 [1:56:45<00:07,  7.01s/it, lr: 1.0e-04 loss: 3.551e-01]



Saved to /content/output/flux_dev_finetune/optimizer.pt


In [ ]:
from google.colab import drive
import shutil
import os

# Mount Google Drive
drive.mount('/content/drive')

source_folder = '/content/output'  


drive_folder = '/content/drive/MyDrive/job_weights/'
os.makedirs(drive_folder, exist_ok=True)  


destination_folder = os.path.join(drive_folder, 'output_weights')
shutil.copytree(source_folder, destination_folder)

print(f"Folder saved to: {destination_folder}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Folder saved to: /content/drive/MyDrive/job_weights/output_weights


In [ ]:
from diffusers import AutoPipelineForText2Image
import torch

# pipeline = AutoPipelineForText2Image.from_pretrained("black-forest-labs/FLUX.1-dev", torch_dtype=torch.float16)

# pipeline = pipeline.to("cuda")

pipeline.load_lora_weights('/content/drive/MyDrive/job_weights/output_weights/flux_dev_finetune', weight_name ='flux_dev_finetune.safetensors')


In [ ]:
prompt = "A Kashmiri shawl-inspired pattern featuring delicate swirls of vines and paisleys in soft pastel shades, woven into an intricate border design for a light, airy, and elegant fabric."

# Generate an image
image = pipeline(prompt).images[0]

# Display the generated image
image.save("/content/drive/MyDrive/ahhh.png")

  0%|          | 0/28 [00:00<?, ?it/s]